##### extra functions

##### those after the predcitions on the image

In [ ]:
output_json_path = "prediction_DENW11AL0000h3Gt.json"

# Prepare the results
threshold = 0.5  # Confidence threshold
image_id = 1  # Assign a unique ID to the image
annotations = []
annotation_id = 0  # Start annotation IDs from 0

# Process predictions
for prediction in predictions:
    boxes = prediction["boxes"].cpu().numpy()  # Convert to NumPy array
    scores = prediction["scores"].cpu().numpy()
    labels = prediction["labels"].cpu().numpy()
    
    for box, score, label in zip(boxes, scores, labels):
        if score >= threshold:
            x_min, y_min, x_max, y_max = map(float, box)  # Ensure native Python floats
            width = x_max - x_min
            height = y_max - y_min
            area = width * height
            score = float(score)
            
            annotation = {
                "id": annotation_id,
                "iscrowd": 0,
                "image_id": image_id,
                "category_id": int(label),
                "bbox": [x_min, y_min, width, height],
                "area": area,
                "score": score
            }
            annotations.append(annotation)
            annotation_id += 1

# Construct the final JSON structure
results = {
    "images": [
        {
            "id": image_id,
            # "width": img_width,
            # "height": img_height,
            "file_name": os.path.basename(image_path)
        }
    ],
    "annotations": annotations
}

# Save results to JSON
with open(output_json_path, "w") as f:
    json.dump(results, f, indent=4)

print(f"Predictions saved to {output_json_path}")

In [ ]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import json
import os

# Specify the target image filename
target_image_filename = "rectified_facade_DENW11AL0000h3Gt.jpg"

# Load ground truth annotations
coco_gt = COCO(os.path.join("Evaluation_subset", "labels_facade_dataset_2024-06-09-08-43-50.json"))

# Get the image ID for the target image
image_id = next((img["id"] for img in coco_gt.dataset["images"] if img["file_name"] == target_image_filename), None)

if image_id is None:
    raise ValueError(f"Image {target_image_filename} not found in the ground truth file.")

# Filter ground truth annotations for the target image
filtered_gt = {
    "images": [img for img in coco_gt.dataset["images"] if img["id"] == image_id],
    "annotations": [ann for ann in coco_gt.dataset["annotations"] if ann["image_id"] == image_id],
    "categories": coco_gt.dataset["categories"],
}

# Save the filtered ground truth to a temporary file
with open("filtered_ground_truth.json", "w") as f:
    json.dump(filtered_gt, f)

# Load the filtered ground truth into COCO
coco_gt_filtered = COCO("filtered_ground_truth.json")

# Load predictions
with open("prediction_DENW11AL0000h3Gt.json", "r") as f:
    predictions = json.load(f)

# Filter predictions for the target image
filtered_predictions = [pred for pred in predictions["annotations"] if pred["image_id"] == image_id]

# Save the filtered predictions to a temporary file
with open("filtered_predictions.json", "w") as f:
    json.dump(filtered_predictions, f)

# Load the filtered predictions into COCO
coco_dt_filtered = coco_gt_filtered.loadRes("filtered_predictions.json")

# Evaluate
coco_eval = COCOeval(coco_gt_filtered, coco_dt_filtered, "bbox")
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()
